<a href="https://colab.research.google.com/github/alejandrolq/StaticalLearningII-Proyecto/blob/main/ProyectoSLII_CNN_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Nombre: José Alejandro López Quel**

**Carné: 21001127**

**Statistical Learning II**

**Sección U**

**Proyecto - Convolutional Neural Network**

In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from keras import Sequential
from keras import backend as K
from keras.models import load_model
from google.colab.patches import cv2_imshow
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator, load_img, array_to_img, img_to_array

In [2]:
path_name_test = '/content/drive/MyDrive/Statistical Learning II/Proyecto/Datasets/CNN_2/Test'
path_name_train = '/content/drive/MyDrive/Statistical Learning II/Proyecto/Datasets/CNN_2/Train'
path_name = '/content/drive/MyDrive/Statistical Learning II/Proyecto/Datasets/CNN'
file_name = 'SpiderData.csv'

In [3]:
data = pd.read_csv(os.path.join(path_name, file_name), encoding= 'unicode_escape', delimiter=";", header=None)

In [4]:
data.head()

,0,1
0,"¡Hola, \n@MaXx_imiliano\n! Gracias por compart...",1.jfif
1,"¡Hola, \n@catalina_parr\n! Gracias por tu cons...",2.jfif
2,"¡Hola, \n@maxxeff4\n! Gracias por compartir. P...",3.jfif
3,"¡Hola, \n@JavierG650ER\n! Gracias por tu consu...",4.jfif
4,"¡Hola, \n@headcrusher666\n! Gracias por tu con...",5.jfif


In [5]:
data[0][0]

'¡Hola, \n@MaXx_imiliano\n! Gracias por compartir. Pertenece a la familia #Araneidae, del género #Neoscona probablemente. Se les conoce como arañas de jardín o de tela orbicular. No son consideradas de importancia médica #NIMWhite heavy check mark'

In [6]:
def format_tweet(column):
  if '#IM' in column:
    return 1
  else:
    return 0

In [7]:
df = data.copy()
df[0] = df[0].apply(format_tweet)
df.head()

,0,1
0,0,1.jfif
1,0,2.jfif
2,0,3.jfif
3,0,4.jfif
4,0,5.jfif


In [8]:
img_size = 224
def get_data(data_dir, df):
  data = [] 
  for index, row in df.iterrows():
    class_num = row[0]
    try:
      img_arr = cv2.imread(os.path.join(data_dir, row[1].strip()))[...,::-1] #convert BGR to RGB format
      resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
      data.append([resized_arr, class_num])
    except Exception as e:
      print(e)
  return np.array(data)

In [9]:
def train_val_split(df, path_name, train_size=0.8, shuffle=True):
  all_images = df.copy()
  if shuffle:
    all_images = all_images.sample(frac=1).reset_index(drop=True)

  train_size = int(len(all_images) * train_size)

  training_data = get_data(path_name, all_images[:train_size])
  validation_data = get_data(path_name, all_images[train_size:])
  return training_data, validation_data

In [10]:
# train = get_data(path_name_train, df[:120])
# val = get_data(path_name_test, df[120:])
train, val = train_val_split(df, path_name)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if sys.path[0] == '':


In [14]:
x_train = []
y_train = []
x_val = []
y_val = []

for feature, label in train:
  x_train.append(feature)
  y_train.append(label)

for feature, label in val:
  x_val.append(feature)
  y_val.append(label)

x_train = np.array(x_train) / 255
x_val = np.array(x_val) / 255

x_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)

x_val.reshape(-1, img_size, img_size, 1)
y_val = np.array(y_val)

In [15]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False)

In [16]:
datagen.fit(x_train)

In [4]:
model = Sequential()
model.add(Conv2D(64, 3, activation="relu", input_shape=(224,224,3)))
model.add(MaxPool2D())

model.add(Conv2D(32, 3, activation="relu"))
model.add(MaxPool2D())

model.add(Conv2D(16, 3, activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(0.45))

model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(2, activation="sigmoid"))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 222, 222, 64)      1792      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 111, 111, 64)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 109, 109, 32)      18464     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 52, 52, 16)        4624      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 26, 26, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 26, 26, 16)       

In [18]:
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer = opt , loss = tf.keras.losses.BinaryCrossentropy(from_logits=True) , metrics = ['accuracy'])

In [19]:
history = model.fit(x_train,y_train,epochs = 10 , validation_data = (x_val, y_val))

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/keras/backend.py:4907: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


4/4 [==============================] - 12s 3s/step - loss: 4.0485 - accuracy: 0.4583 - val_loss: 0.6722 - val_accuracy: 0.9000
Epoch 2/10
4/4 [==============================] - 10s 2s/step - loss: 0.6198 - accuracy: 0.7833 - val_loss: 0.6317 - val_accuracy: 0.9000
Epoch 3/10
4/4 [==============================] - 10s 2s/step - loss: 0.6177 - accuracy: 0.7833 - val_loss: 0.6353 - val_accuracy: 0.9000
Epoch 4/10
4/4 [==============================] - 10s 2s/step - loss: 0.6566 - accuracy: 0.7833 - val_loss: 0.6038 - val_accuracy: 0.9000
Epoch 5/10
4/4 [==============================] - 10s 2s/step - loss: 0.6221 - accuracy: 0.7833 - val_loss: 0.5668 - val_accuracy: 0.9000
Epoch 6/10
4/4 [==============================] - 10s 2s/step - loss: 1.2375 - accuracy: 0.7833 - val_loss: 0.4842 - val_accuracy: 0.9000
Epoch 7/10
4/4 [==============================] - 10s 2s/step - loss: 0.5692 - accuracy: 0.7833 - val_loss: 0.4961 - val_accuracy: 0.9000
Epoch 8/10
4/4 [=============================

In [20]:
model.compile(optimizer = 'adam' , loss='binary_crossentropy' , metrics = ['accuracy'])

In [21]:
history = model.fit(x_train, y_train, epochs = 10, steps_per_epoch=5, validation_data = (x_val, y_val))

Epoch 1/10
5/5 [==============================] - 10s 2s/step - loss: 0.5255 - accuracy: 0.7833 - val_loss: 0.3904 - val_accuracy: 0.9000
Epoch 2/10
5/5 [==============================] - 10s 2s/step - loss: 0.5243 - accuracy: 0.7833 - val_loss: 0.3884 - val_accuracy: 0.9000
Epoch 3/10
5/5 [==============================] - 10s 2s/step - loss: 0.5239 - accuracy: 0.7833 - val_loss: 0.3867 - val_accuracy: 0.9000
Epoch 4/10
5/5 [==============================] - 10s 2s/step - loss: 0.5237 - accuracy: 0.7833 - val_loss: 0.3852 - val_accuracy: 0.9000
Epoch 5/10
5/5 [==============================] - 10s 2s/step - loss: 0.5234 - accuracy: 0.7833 - val_loss: 0.3832 - val_accuracy: 0.9000
Epoch 6/10
5/5 [==============================] - 10s 2s/step - loss: 0.5233 - accuracy: 0.7833 - val_loss: 0.3813 - val_accuracy: 0.9000
Epoch 7/10
5/5 [==============================] - 10s 2s/step - loss: 0.5231 - accuracy: 0.7833 - val_loss: 0.3804 - val_accuracy: 0.9000
Epoch 8/10
5/5 [==================

In [22]:
y_train

array([0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0])